## Initialization

In [ ]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

## Clean Version

### Louvain Baseline (idx: 0)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 0
    data_for_excel['method_name'] = 'Louv_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))
                
                data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '100%', Num_Cluster_Louv_on_G]
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden baseline (idx: 1)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 1
    data_for_excel['method_name'] = 'Leiden_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_, Purity_ = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))
                
                data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '100%', Num_Cluster_Leiden_on_G]
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Majority vote (2)

In [5]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 2
    data_for_excel['method_name'] = 'Leiden + Vote'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%' 
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.791
59910
[0 1 2 3 4 5 6 7 8 9]
(11235, 19939)
selected vertices  11235
(11235,) Counter({'TCGA-BRCA': 1226, 'TCGA-KIRC': 610, 'TCGA-LUAD': 589, 'TCGA-UCEC': 585, 'TCGA-THCA': 572, 'TCGA-HNSC': 566, 'TCGA-PRAD': 554, 'TCGA-LUSC': 552, 'TCGA-LGG': 534, 'TCGA-COAD': 514, 'TCGA-SKCM': 473, 'TCGA-STAD': 448, 'TCGA-OV': 429, 'TCGA-BLCA': 428, 'TCGA-LIHC': 424, 'TCGA-KIRP': 323, 'TCGA-CESC': 309, 'TCGA-SARC': 265, 'TCGA-ESCA': 198, 'TCGA-PCPG': 187, 'TCGA-PAAD': 183, 'TCGA-READ': 177, 'TCGA-GBM': 174, 'TCGA-TGCT': 156, 'TCGA-LAML': 151, 'TCGA-THYM': 122, 'TCGA-KICH': 91, 'TCGA-MESO': 87, 'TCGA-UVM': 80, 'TCGA-ACC': 79, 'TCGA-UCS': 57, 'TCGA-DLBC': 48, 'TCGA-CHOL': 44})
33 32
[0 1 2 3 4 5 6 7 8 9]
(11020, 1882)
selected vertices  11020
(11020,) Counter({'TCGA-BRCA': 1202, 'TCGA-KIRC': 592, 'TCGA-UCEC': 575, 'TCGA-THCA': 573, 'TCGA-HNSC': 569, 'TCGA-LUAD': 564, 'TCGA-PRAD': 551, 'TCGA-LGG': 530, 'TCGA-LUSC': 523, 'TC

C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


20 19
biorxiv (68131, 50)
26 25
big_patent (17064, 50)
9 8
7 6
70 69


i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


Validation for `citeseer_labels` and `citeseer_features` passes.
7 6


i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


42 41
ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.701
151020
AMB 12832
Log transform done
(12832, 50)
Accuracy of  15 -NN graph is 0.764
192480
Baron_Human 8569
Log transform done
(8569, 50)
Accuracy of  15 -NN graph is 0.972
128535
Baron_Mouse 1886
Log transform done
(1886, 50)
Accuracy of  15 -NN graph is 0.954
28290
Muraro 2122
Log transform done
(2122, 50)
Accuracy of  15 -NN graph is 0.951
31830
Segerstolpe 2133
Log transform done
(2133, 50)
Accuracy of  15 -NN graph is 0.932
31995
Tcell-medicine 5759
Log transform done
(5759, 50)
Accuracy of  15 -NN graph is 0.735
86385
TM 54865
Log transform done
(54865, 50)
Accuracy of  15 -NN graph is 0.949
822975
VISP 15413
Log transform done
(15413, 50)
Accuracy of  15 -NN graph is 0.709
231195
Xin 1449
Log transform done
(1449, 50)
Accuracy of  15 -NN graph is 0.975
21735
Zheng 65943
Log transform done
(65943, 50)
Accuracy of  15 -NN graph is 0.561
989145


### Leiden + Majority vote + Leiden (3)

In [6]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 3
    data_for_excel['method_name'] = 'Leiden + Vote + Leiden'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%' 
                    
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.792
59910
[0 1 2 3 4 5 6 7 8 9]
(11235, 19939)
selected vertices  11235
(11235,) Counter({'TCGA-BRCA': 1226, 'TCGA-KIRC': 610, 'TCGA-LUAD': 589, 'TCGA-UCEC': 585, 'TCGA-THCA': 572, 'TCGA-HNSC': 566, 'TCGA-PRAD': 554, 'TCGA-LUSC': 552, 'TCGA-LGG': 534, 'TCGA-COAD': 514, 'TCGA-SKCM': 473, 'TCGA-STAD': 448, 'TCGA-OV': 429, 'TCGA-BLCA': 428, 'TCGA-LIHC': 424, 'TCGA-KIRP': 323, 'TCGA-CESC': 309, 'TCGA-SARC': 265, 'TCGA-ESCA': 198, 'TCGA-PCPG': 187, 'TCGA-PAAD': 183, 'TCGA-READ': 177, 'TCGA-GBM': 174, 'TCGA-TGCT': 156, 'TCGA-LAML': 151, 'TCGA-THYM': 122, 'TCGA-KICH': 91, 'TCGA-MESO': 87, 'TCGA-UVM': 80, 'TCGA-ACC': 79, 'TCGA-UCS': 57, 'TCGA-DLBC': 48, 'TCGA-CHOL': 44})
33 32
[0 1 2 3 4 5 6 7 8 9]
(11020, 1882)
selected vertices  11020
(11020,) Counter({'TCGA-BRCA': 1202, 'TCGA-KIRC': 592, 'TCGA-UCEC': 575, 'TCGA-THCA': 573, 'TCGA-HNSC': 569, 'TCGA-LUAD': 564, 'TCGA-PRAD': 551, 'TCGA-LGG': 530, 'TCGA-LUSC': 523, 'TC

C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


20 19
biorxiv (68131, 50)
26 25
big_patent (17064, 50)
9 8
7 6
70 69


i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


Validation for `citeseer_labels` and `citeseer_features` passes.
7 6


i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


42 41
ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.702
151020
AMB 12832
Log transform done
(12832, 50)
Accuracy of  15 -NN graph is 0.762
192480
Baron_Human 8569
Log transform done
(8569, 50)
Accuracy of  15 -NN graph is 0.972
128535
Baron_Mouse 1886
Log transform done
(1886, 50)
Accuracy of  15 -NN graph is 0.954
28290
Muraro 2122
Log transform done
(2122, 50)
Accuracy of  15 -NN graph is 0.951
31830
Segerstolpe 2133
Log transform done
(2133, 50)
Accuracy of  15 -NN graph is 0.932
31995
Tcell-medicine 5759
Log transform done
(5759, 50)
Accuracy of  15 -NN graph is 0.737
86385
TM 54865
Log transform done
(54865, 50)
Accuracy of  15 -NN graph is 0.949
822975
VISP 15413
Log transform done
(15413, 50)
Accuracy of  15 -NN graph is 0.711
231195
Xin 1449
Log transform done
(1449, 50)
Accuracy of  15 -NN graph is 0.974
21735
Zheng 65943
Log transform done
(65943, 50)
Accuracy of  15 -NN graph is 0.562
989145


### Leiden + Vote + Leiden + Vote (4)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 4
    data_for_excel['method_name'] = '(Leiden + Vote)x2'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    
                    
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    '''
                    Leiden + vote + Leiden + vote
                    '''
                    nodes_rest2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ == -1]
                    NMI_List2, Purity_List2, Balance_List2, Preserv_List2, InEdge_List2 = merge_by_vote(selected_node_list, nodes_rest2, H_label2, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num2 = len([node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ != -1])
                    selected_node_percent = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%' 

                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Strong Majority Vote + Random Walk (Weak Majority Vote) (idx: 5)

### Freeze pane

In [ ]:
from subgraph import *
freeze_panes('results.xlsx')

: 

## Leiden + Strong Majority Vote + Random Walk on unselected nodes

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05,0.25, 0.5, 1, 1.5, 5]

for data in data_names: 
    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/(Vote+RandWalk)'+data+'.pdf'
    data_for_plot = defaultdict(lambda: defaultdict(dict)) #nested dictionary to store data for plots

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)
    print('Number of nodes:',G.number_of_nodes())

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))

                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    # list of unselected_nodes
                    unselected_nodes = [node_idx for node_idx, node_label in enumerate(H_label) if node_label == -1]
                    selected_node_num = G.number_of_nodes() - len(unselected_nodes)
                    '''
                    Random walks from unselected nodes
                    '''
                    H_label, NMI_rand_walk, Purity_rand_walk = do_random_walks(G, unselected_nodes, label, H_label, node2FR) 
                    selected_node_num2 = len([x for x in H_label if x != -1])
                    #print('selected_before_randwalk:',selected_node_num, 'selected_after_randwalk:',selected_node_num2)
                    '''
                    Plots
                    '''
                    #at index [res][FR_type] store the data for the plot
                    data_for_plot[idx][FR_type] = {'NMI_after_vote':NMI_List[-1], 'Purity_after_vote':Purity_List[-1], 'Louvain_NMI':NMI_, 
                                                    'Louvain_Purity':Purity_, 'Leiden_NMI':NMI_baseline_leiden, 'Leiden_Purity':Purity_baseline_leiden, 
                                                    'Num_Cluster_Louv_on_G':Num_Cluster_Louv_on_G, 'Num_Cluster_Leiden_on_G':Num_Cluster_Leiden_on_G, 
                                                    'Num_Cluster_Louv_on_H':Num_Cluster_Louv_on_H, 'percent_nodes':(selected_node_num*100 / G.number_of_nodes()),
                                                    'extra_nmi':NMI_rand_walk, 'extra_purity':Purity_rand_walk,
                                                    'extra_name':'+RandWalk', 'extra_percent_nodes':(selected_node_num2*100 / G.number_of_nodes())
                                                    }
                    
    plot_to_pdf(resolution_list, data_for_plot, k, True_num_of_clusters, pdf_name, data)
    
        
            

                                
                    

                    
                    

